In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from finance_byu.summarize import summary

c:\Users\benja\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
crsp_daily1 = pd.read_feather('C:/Users/benja/desktop/ACME_Senior/Fin585/Final Project/crsp_daily.ftr')

crsp_monthly1 = pd.read_feather('C:/Users/benja/desktop/ACME_Senior/Fin585/Final Project/crsp_monthly.ftr')

In [27]:
# for both datasets keep only the data post jan 1 2000 from caldt
crsp_daily2 = crsp_daily1[(crsp_daily1['caldt'] >= '1980-01-01') & (crsp_daily1['caldt'] <= '2007-12-31')]
crsp_monthly2 = crsp_monthly1[(crsp_monthly1['caldt'] >= '1980-01-01') & (crsp_monthly1['caldt'] <= '2007-12-31')]

In [243]:
crsp_daily = crsp_daily2.copy()
crsp_monthly = crsp_monthly2.copy()

In [244]:
# Cleaning the daily dataset
crsp_daily['prc'] = abs(crsp_daily['prc'])
crsp_daily['prc_lag'] = crsp_daily.groupby('permno')['prc'].shift(1)
# crsp_daily = crsp_daily[crsp_daily['ret'] > -1]
crsp_daily = crsp_daily[crsp_daily['prc_lag'] > 5]
crsp_daily.sort_values(by = ['permno', 'caldt'], inplace = True)
crsp_daily.drop(columns = ['shrcd', 'excd', 'siccd', 'vol', 'shr', 'prc_lag'], inplace = True)

# Adding column for positive and negative returns
crsp_daily['ret_class'] = np.where(crsp_daily['ret'].shift(1) >= 0, '1', '0')

# Calculate rolling yearly number of positive and negative days for each stock
n = 250
crsp_daily['pos_days'] = crsp_daily.groupby('permno').rolling(window = n, min_periods = n)['ret_class'].sum().reset_index(level=0, drop=True)
crsp_daily['neg_days'] = n - crsp_daily['pos_days']
crsp_daily['%pos'] = crsp_daily['pos_days'] / n
crsp_daily['%neg'] = crsp_daily['neg_days'] / n
crsp_daily['%neg - %pos'] = crsp_daily['%neg'] - crsp_daily['%pos']
crsp_daily.drop(columns = ['ret_class', 'pos_days', 'neg_days', '%pos', '%neg'], inplace = True)
crsp_daily.rename(columns = {'ret': 'daily_ret'}, inplace = True)
crsp_daily.dropna(inplace = True)

In [245]:
crsp_monthly = crsp_monthly2.copy()
# crsp_daily.head()

In [246]:
crsp_monthly.drop(columns = ['cusip', 'ticker', 'shrcd', 'excd', 'siccd', 'vol', 'shr', 'cumfacshr'], inplace = True)

crsp_monthly['prc'] = abs(crsp_monthly['prc'])
crsp_monthly['prc_lag'] = crsp_monthly.groupby('permno')['prc'].shift(1)
crsp_monthly = crsp_monthly[crsp_monthly['prc_lag'] > 5]

crsp_monthly['monthly_ret_lag'] = crsp_monthly.groupby('permno')['ret'].shift(1) 

# keep ret >-1
# crsp_monthly = crsp_monthly[crsp_monthly['ret'] > -1]
crsp_monthly.rename(columns = {'ret': 'monthly_ret'}, inplace = True)
# crsp_monthly.head()

In [247]:
crsp_monthly['logret'] = np.log(1 + crsp_monthly['monthly_ret_lag'])
crsp_monthly['mom'] = crsp_monthly.groupby('permno')['logret'].rolling(11,11).sum().reset_index(drop=True, level=0)
crsp_monthly['mom'] = crsp_monthly.groupby('permno')['mom'].shift(2)
crsp_monthly.dropna(inplace=True)
# crsp_monthly.tail()

In [248]:
# I want to merge the two datasets on permno and caldt and keep everymonthly observation
crsp = pd.merge(crsp_daily, crsp_monthly, on = ['permno', 'caldt'])

# crsp = pd.merge(crsp_daily, crsp_monthly, on = ['permno', 'caldt'], how = 'left')

#fill any nans with the data that follows
# crsp.fillna(method = 'bfill', inplace = True)

crsp.drop(columns = ['logret', 'prc_x', 'prc_y'], inplace = True)

# crsp

In [249]:
crsp['id'] = np.sign(crsp['mom']) * crsp['%neg - %pos']
crsp.head()

,permno,caldt,daily_ret,date,%neg - %pos,monthly_ret,prc_lag,monthly_ret_lag,mom,id
0,10001,1987-03-31,0.085106,1987-03-31,-0.488,0.036800,6.2500,-0.074074,0.196692,-0.488
1,10001,1987-04-30,0.042553,1987-04-30,-0.432,-0.039216,6.3750,0.036800,0.140122,-0.432
2,10001,1987-05-29,-0.031915,1987-05-29,-0.384,-0.071429,6.1250,-0.039216,0.038273,-0.384
3,10001,1987-06-30,0.068182,1987-06-30,-0.352,0.051429,5.6875,-0.071429,0.064560,-0.352
4,10001,1987-07-31,0.000000,1987-07-31,-0.320,0.021277,5.8750,0.051429,0.034407,-0.320


In [250]:
crsp['mom_bins'] = crsp.groupby('caldt')['mom'].transform(pd.qcut,2,labels=False)
# crsp.tail()

In [118]:
crsp['id_bins'] = crsp.groupby(["caldt", "mom_bins"])['id'].transform(pd.qcut,5, labels=False)
# crsp.tail()

In [119]:
crsp['id_bins2'] = crsp.groupby(["caldt"])['id'].transform(pd.qcut,5, labels=False)
# crsp.tail()

In [124]:
port = crsp.groupby(['caldt','mom_bins','id_bins'])['monthly_ret'].mean()*100
port = port.unstack(level=['mom_bins','id_bins'])
# port.head()

In [125]:
port1 = crsp.groupby(['caldt','mom_bins','id_bins2'])['monthly_ret'].mean()*100
port1 = port1.unstack(level=['mom_bins','id_bins2'])
# port1.head()

In [126]:
summary(port).loc[['mean','std','tstat']].round(3)

mom_bins      0                                    1                        \
id_bins       0      1      2      3       4       0       1      2      3   
mean      0.959  1.022  1.114  1.647   2.146   2.339   2.396  1.768  1.002   
std       4.472  5.195  5.152  4.785   3.642   3.082   4.186  4.633  5.093   
tstat     3.849  3.530  3.879  6.176  10.575  13.620  10.273  6.848  3.530   

mom_bins         
id_bins       4  
mean     -0.221  
std       5.545  
tstat    -0.716

In [127]:
summary(port1).loc[['mean','std','tstat']].round(3)

mom_bins      0                                   1                       \
id_bins2      0      1      2      3      4       0      1      2      3   
mean      0.824  0.815  0.907  0.965  1.919   2.435  1.946  0.941 -0.365   
std       4.762  5.361  5.211  5.206  4.035   3.358  4.475  5.043  6.055   
tstat     2.928  2.711  3.123  3.326  8.532  13.011  7.801  3.349 -1.070   

mom_bins         
id_bins2      4  
mean     -1.680  
std       8.971  
tstat    -2.661

In [324]:
returns = crsp_monthly2.copy() 
returns.drop(columns = ['cusip', 'ticker','shrcd', 'excd', 'siccd', 'vol', 'shr', 'cumfacshr'], inplace = True)

returns['prc'] = abs(returns['prc'])
returns['prc_lag'] = returns.groupby('permno')['prc'].shift(1)
returns = returns[returns['prc_lag'] > 5]
returns.dropna(inplace = True)

returns.head()

,permno,caldt,prc,ret,prc_lag
18,10001,1986-02-28,6.2500,0.020408,6.1250
19,10001,1986-03-31,6.3125,0.025200,6.2500
20,10001,1986-04-30,6.3750,0.009901,6.3125
21,10001,1986-05-30,6.3125,-0.009804,6.3750
22,10001,1986-06-30,6.1250,-0.013069,6.3125


In [325]:
# Calculate formation period returns (PRET) for each stock
# Example: Assuming 'returns' is a DataFrame with columns ['Date', 'Ticker', 'Return']
# Calculate PRET over a 12-month formation period
returns['logret'] = np.log(1 + returns['ret'])
returns['mom'] = returns.groupby('permno')['logret'].rolling(11,11).sum().reset_index(drop=True, level=0)
returns['mom'] = returns.groupby('permno')['mom'].shift(2)
returns.dropna(inplace=True)
returns.head()

,permno,caldt,prc,ret,prc_lag,logret,mom
30,10001,1987-02-27,6.2500,-0.074074,6.7500,-0.076961,0.196692
31,10001,1987-03-31,6.3750,0.036800,6.2500,0.036139,0.140122
32,10001,1987-04-30,6.1250,-0.039216,6.3750,-0.040006,0.038273
33,10001,1987-05-29,5.6875,-0.071429,6.1250,-0.074108,0.064560
34,10001,1987-06-30,5.8750,0.051429,5.6875,0.050150,0.034407


In [326]:
# sort the data by permno and date
returns.sort_values(by = ['permno', 'caldt'], inplace = True)

# Adding column for positive and negative returns
returns['ret_class'] = np.where(returns['ret'].shift(1) >= 0, '1', '0')

# Calculate rolling yearly number of positive and negative days for each stock
n = 24
returns['pos_days'] = returns.groupby('permno').rolling(window = n, min_periods = n)['ret_class'].sum().reset_index(level=0, drop=True)
returns['neg_days'] = n - returns['pos_days']
returns['%pos'] = returns['pos_days'] / n
returns['%neg'] = returns['neg_days'] / n
returns['%neg - %pos'] = returns['%neg'] - returns['%pos']
returns.drop(columns = ['ret_class', 'pos_days', 'neg_days', '%pos', '%neg', 'prc', 'prc_lag'], inplace = True)
# returns.rename(columns = {'ret': 'm_ret'}, inplace = True)
returns.dropna(inplace = True)
returns.head()

,permno,caldt,ret,logret,mom,%neg - %pos
53,10001,1989-01-31,0.019608,0.019418,0.172499,-0.083333
54,10001,1989-02-28,0.038462,0.037741,0.089265,-0.166667
55,10001,1989-03-31,0.017778,0.017622,0.031722,-0.250000
56,10001,1989-04-28,0.074074,0.071459,0.148827,-0.250000
57,10001,1989-05-31,-0.034483,-0.035092,0.136296,-0.333333


In [327]:
# Calculate ID measure
returns['ID'] = np.sign(returns['mom']) * (returns['%neg - %pos'])
returns.head()

,permno,caldt,ret,logret,mom,%neg - %pos,ID
53,10001,1989-01-31,0.019608,0.019418,0.172499,-0.083333,-0.083333
54,10001,1989-02-28,0.038462,0.037741,0.089265,-0.166667,-0.166667
55,10001,1989-03-31,0.017778,0.017622,0.031722,-0.250000,-0.250000
56,10001,1989-04-28,0.074074,0.071459,0.148827,-0.250000,-0.250000
57,10001,1989-05-31,-0.034483,-0.035092,0.136296,-0.333333,-0.333333


In [328]:
returns['mom_bins'] = returns.groupby('caldt')['mom'].transform(pd.qcut,2,labels=False)
returns['id_bins'] = returns.groupby(["caldt", "mom_bins"])['ID'].transform(pd.qcut,5, labels=False)
returns['id_bins2'] = returns.groupby(["caldt"])['ID'].transform(pd.qcut,5, labels=False)

In [337]:
port = returns.groupby(['caldt','mom_bins','id_bins'])['ret'].mean()*100
port = port.unstack(level=['mom_bins','id_bins'])

In [338]:
port1 = returns.groupby(['caldt','mom_bins','id_bins2'])['ret'].mean()*100
port1 = port1.unstack(level=['mom_bins','id_bins2'])

In [339]:
summary(port).loc[['mean','std','tstat']].round(3)

mom_bins      0                                  1                            
id_bins       0      1      2      3      4      0      1      2      3      4
mean      1.067  0.919  0.783  0.790  0.856  1.533  1.516  1.495  1.389  1.278
std       4.381  4.658  4.614  4.626  4.295  3.885  3.986  4.051  4.239  4.607
tstat     4.221  3.418  2.939  2.957  3.451  6.833  6.588  6.389  5.673  4.803

In [340]:
summary(port1).loc[['mean','std','tstat']].round(3)

mom_bins      0                                  1                            
id_bins2      0      1      2      3      4      0      1      2      3      4
mean      1.104  0.960  0.891  0.786  0.836  1.507  1.474  1.461  1.347  1.184
std       4.311  4.546  4.629  4.674  4.409  3.939  4.061  4.239  4.485  4.820
tstat     4.437  3.659  3.333  2.913  3.284  6.628  6.286  5.969  5.202  4.255